#Immoscout24 Preparing data for SPSS


## Preparation:

In [0]:
from pandas import DataFrame, read_csv, read_excel
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
import scipy
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import os
%matplotlib inline

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
DF = pd.read_excel("/content/drive/My Drive/Datasets/LIWC_Immoscout_complete.xlsx", sep='\t', error_bad_lines=False, dtype='unicode', lineterminator='\n')
pd.options.display.max_columns = None

# Extracting informal features 

### Creating categories of various variables to use for analysis

#### Constructionyear

In [0]:
DF2 = DF
DF2.Baujahr = pd.to_numeric(DF2.Baujahr, errors='coerce').fillna(0).astype(np.int64)
DF2['constructionyear_range'] = pd.qcut(DF2['Baujahr'], 5, labels=["very old", "old", "medium", "new", "very new"])
col = "constructionyear_range" 
DF2 = pd.concat([DF2[col],DF2.drop(col,axis=1)], axis=1)
DF2

,constructionyear_range,ID,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,Stadt,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r
0,very new,115756801,Bungalow,518500.0,A+,Erstbezug,3.0,160.0,1200.0,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,Ranstadt,4,NaN,NaN,NaN,NaN,NaN,NaN,0,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r
1,very old,116853306,Einfamilienhaus (freistehend),301667.0,NaN,Erstbezug,4.0,135.0,860.0,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,Nideggen,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,12,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
2,very new,116959865,Einfamilienhaus (freistehend),225900.0,A+,Erstbezug,3.0,76.0,612.0,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,04523,Pegau,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,24,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r
3,very new,116953174,Einfamilienhaus (freistehend),218659.0,NaN,Erstbezug,6.0,144.0,550.0,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,52428,12,NaN,NaN,NaN,NaN,NaN,NaN,10,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r
4,very old,116953218,Einfamilienhaus (freistehend),287200.0,NaN,Erstbezug,4.0,141.0,801.0,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,Storkow,12,NaN,NaN,NaN,NaN,NaN,NaN,10,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.16,0.00,0.00,0.00,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19.77,18.60,1.16,0.00,0.00,0.00,0.00,0.00,0.00,5.81,1.16,2.33,1.16,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

#### Make column with avg price per m2 per city

In [0]:
DF3 = DF2
DF3.Kaufpreis = pd.to_numeric(DF3.Kaufpreis, errors='coerce').fillna(0).astype(np.int64)
DF3['Wohnflaeche (in m²)'] = pd.to_numeric(DF3['Wohnflaeche (in m²)'], errors='coerce').fillna(0).astype(np.int64)
DF3['price_m2'] = DF3.Kaufpreis / DF3['Wohnflaeche (in m²)']
DF3['price_m2'] = DF3['price_m2'].replace([np.inf, -np.inf], np.nan)
DF3.price_m2 = pd.to_numeric(DF3.price_m2, errors='coerce').fillna(0).astype(np.int64)
col = "price_m2" 
DF3 = pd.concat([DF3[col],DF3.drop(col,axis=1)], axis=1)


In [0]:
DF4 = DF3
mean_col = DF4.groupby(['Stadt'])['price_m2'].mean() # not resetting the index!
DF4 = DF4.set_index(['Stadt']) # make the same index here
DF4['avg_price_m2_per_city'] = mean_col
DF4 = DF4.reset_index() # to take the hierarchical index off again
# DF4 = DF4.drop('avg_price_m2', 1)
DF4.avg_price_m2_per_city = pd.to_numeric(DF4.avg_price_m2_per_city, errors='coerce').fillna(0).astype(np.int64)
DF4 = pd.concat([DF4['avg_price_m2_per_city'],DF4.drop('avg_price_m2_per_city',axis=1)], axis=1)
DF4

,avg_price_m2_per_city,Stadt,price_m2,constructionyear_range,ID,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r
0,3598,Ranstadt,3240,very new,115756801,Bungalow,518500,A+,Erstbezug,3.0,160,1200.0,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,NaN,NaN,0,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r
1,1821,Nideggen,2234,very old,116853306,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4.0,135,860.0,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,12,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
2,1773,Pegau,2972,very new,116959865,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3.0,76,612.0,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,04523,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,24,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r
3,1518,52428,1518,very new,116953174,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6.0,144,550.0,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,NaN,NaN,10,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r
4,2260,Storkow,2036,very old,116953218,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4.0,141,801.0,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,NaN,NaN,10,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.16,0.00,0.00,0.00,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19.77,18.60,1.16,0.00,0.00,0.00,0.00,0.00,0.00,5.81,1.16,2.33,1.16,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

#### Make column with category if avg price is higher or lower than avg price of city

- if avg price >120% then label "expensive"
- if average price within 80% - 120% then label "normal"
- if average price <80% then label "cheap"

In [0]:
DF5 = DF4

In [0]:
DF5['perc_price_of_avg'] = 100*DF5['price_m2']/DF5['avg_price_m2_per_city']
col = "perc_price_of_avg" 
DF5 = pd.concat([DF5[col],DF5.drop(col,axis=1)], axis=1)
DF5.perc_price_of_avg = pd.to_numeric(DF5.perc_price_of_avg, errors='coerce').fillna(0).astype(np.int64)

In [0]:
DF5['price_class'] = pd.cut(DF5.perc_price_of_avg, [0,80,120,200], labels = ['cheap', 'normal', 'expensive'], include_lowest=True)
DF5 = pd.concat([DF5['price_class'],DF5.drop('price_class',axis=1)], axis=1)


In [0]:
DF5['price_class_num'] = pd.cut(DF5.perc_price_of_avg, [0,80,120,200], labels = ['1', '2', '3'], include_lowest=True)
DF5 = pd.concat([DF5['price_class_num'],DF5.drop('price_class_num',axis=1)], axis=1)
DF5

,price_class_num,price_class,perc_price_of_avg,avg_price_m2_per_city,Stadt,price_m2,constructionyear_range,ID,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r
0,2,normal,90,3598,Ranstadt,3240,very new,115756801,Bungalow,518500,A+,Erstbezug,3.0,160,1200.0,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,NaN,NaN,0,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r
1,3,expensive,122,1821,Nideggen,2234,very old,116853306,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4.0,135,860.0,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,12,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
2,3,expensive,167,1773,Pegau,2972,very new,116959865,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3.0,76,612.0,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,04523,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,24,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r
3,2,normal,100,1518,52428,1518,very new,116953174,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6.0,144,550.0,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,NaN,NaN,10,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r
4,2,normal,90,2260,Storkow,2036,very old,116953218,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4.0,141,801.0,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,NaN,NaN,10,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.16,0.00,0.00,0.00,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19.77,18.60,1.16,0.00,0.00,0.00,0.00,0.00,0.00,5.81,1.16,2.33,1.16,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [0]:
DF5.price_class_num = pd.to_numeric(DF5.price_class_num, errors='coerce').fillna(0).astype(np.int64)

### Dummifiying topics

In [0]:
DummyDF = DF5
DummyDF = pd.concat([DummyDF, pd.get_dummies(DummyDF['Topic'])], axis=1)
DummyDF = DummyDF.set_index('ID')
DummyDF.drop(['0', '12', '24', '10', '14', '19', '11', '9', '18', '17', '21', '15', '20', '23', '6', '8'], axis=1, inplace=True)

DummyDF


,price_class_num,price_class,perc_price_of_avg,avg_price_m2_per_city,Stadt,price_m2,constructionyear_range,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r,courtyard_estate,garage,house_auction,house_layout,house_viewing,multi_dwelling,renovated
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
115756801,2,normal,90,3598,Ranstadt,3240,very new,Bungalow,518500,A+,Erstbezug,3.0,160,1200.0,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,NaN,NaN,0,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r,0,0,0,0,0,0,0
116853306,3,expensive,122,1821,Nideggen,2234,very old,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4.0,135,860.0,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,12,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r,0,0,0,0,0,0,0
116959865,3,expensive,167,1773,Pegau,2972,very new,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3.0,76,612.0,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,04523,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,24,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r,0,0,0,0,0,0,0
116953174,2,normal,100,1518,52428,1518,very new,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6.0,144,550.0,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,NaN,NaN,10,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r,0,0,0,0,0,0,0
116953218,2,normal,90,2260,Storkow,2036,very old,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4.0,141,801.0,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,NaN,NaN,10,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.16,0.00,0.00,0.00,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19.77,18.60,1.16,0.00,0.00,0.00,0.00,0.00,0

## Find out which descriptions contains "quiet" or similar to then find patterns of quiet houses

In [0]:
mylist = ['ruhig', 'ruhige Lage', 'still', 'Ruhe', 'kein Lärm']
pattern = '|'.join(mylist)
pattern = '|'.join([f'(?i){quiet}' for quiet in mylist]) 

pattern


'(?i)ruhig|(?i)ruhige Lage|(?i)still|(?i)Ruhe|(?i)kein Lärm'

In [0]:
DummyDF['quiet_property'] = DummyDF.Beschreibung.str.contains(pattern)
DummyDF = pd.concat([DummyDF['quiet_property'],DummyDF.drop('quiet_property',axis=1)], axis=1)
DummyDF

,quiet_property,price_class_num,price_class,perc_price_of_avg,avg_price_m2_per_city,Stadt,price_m2,constructionyear_range,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r,courtyard_estate,garage,house_auction,house_layout,house_viewing,multi_dwelling,renovated
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
115756801,False,2,normal,90,3598,Ranstadt,3240,very new,Bungalow,518500,A+,Erstbezug,3.0,160,1200.0,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,NaN,NaN,0,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r,0,0,0,0,0,0,0
116853306,False,3,expensive,122,1821,Nideggen,2234,very old,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4.0,135,860.0,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,12,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r,0,0,0,0,0,0,0
116959865,False,3,expensive,167,1773,Pegau,2972,very new,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3.0,76,612.0,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,04523,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,24,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r,0,0,0,0,0,0,0
116953174,False,2,normal,100,1518,52428,1518,very new,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6.0,144,550.0,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,NaN,NaN,10,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r,0,0,0,0,0,0,0
116953218,False,2,normal,90,2260,Storkow,2036,very old,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4.0,141,801.0,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,NaN,NaN,10,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.16,0.00,0.00,0.00,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.

## Find descriptions mentioning "noisy" features 

In [0]:
mylist = ['laut', 'laute Umgebung', 'Lärm']
pattern2 = '|'.join(mylist)
pattern2 = '|'.join([f'(?i){word}' for word in mylist]) 
DummyDF['noisy_property'] = DummyDF.Beschreibung.str.contains(pattern2)
DummyDF = pd.concat([DummyDF['noisy_property'],DummyDF.drop('noisy_property',axis=1)], axis=1)
DummyDF

,noisy_property,quiet_property,price_class_num,price_class,perc_price_of_avg,avg_price_m2_per_city,Stadt,price_m2,constructionyear_range,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r,courtyard_estate,garage,house_auction,house_layout,house_viewing,multi_dwelling,renovated
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
115756801,False,False,2,normal,90,3598,Ranstadt,3240,very new,Bungalow,518500,A+,Erstbezug,3.0,160,1200.0,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,NaN,NaN,0,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r,0,0,0,0,0,0,0
116853306,False,False,3,expensive,122,1821,Nideggen,2234,very old,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4.0,135,860.0,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,12,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r,0,0,0,0,0,0,0
116959865,False,False,3,expensive,167,1773,Pegau,2972,very new,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3.0,76,612.0,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,04523,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,24,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r,0,0,0,0,0,0,0
116953174,False,False,2,normal,100,1518,52428,1518,very new,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6.0,144,550.0,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,NaN,NaN,10,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r,0,0,0,0,0,0,0
116953218,False,False,2,normal,90,2260,Storkow,2036,very old,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4.0,141,801.0,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,NaN,NaN,10,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.16,0.00,0.00,0.00,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3

In [0]:
DummyDF['noisy_property'].value_counts()

False    51405
True       974
Name: noisy_property, dtype: int64

## Find descriptions mentioning "garden"

In [0]:
mylist = ['Garten', 'garten', 'eigener Garten', 'Gemeinschaftsgarten']
pattern3 = '|'.join(mylist)
pattern3 = '|'.join([f'(?i){word}' for word in mylist]) 
DummyDF['garden'] = DummyDF.Beschreibung.str.contains(pattern3)
DummyDF = pd.concat([DummyDF['garden'],DummyDF.drop('garden',axis=1)], axis=1)

In [0]:
DummyDF['garden'].value_counts()

True     26358
False    26021
Name: garden, dtype: int64

## Find descriptions mentioning "light"

In [0]:
mylist = ['Lichtdurchflutet', 'natürliches Licht', 'viel Tageslicht', 'helles Wohnzimmer', 'hell', 'viel Licht']
pattern4 = '|'.join(mylist)
pattern4 = '|'.join([f'(?i){word}' for word in mylist]) 
DummyDF['light_flooded'] = DummyDF.Beschreibung.str.contains(pattern4)
DummyDF = pd.concat([DummyDF['light_flooded'],DummyDF.drop('light_flooded',axis=1)], axis=1)
DummyDF

,light_flooded,garden,noisy_property,quiet_property,price_class_num,price_class,perc_price_of_avg,avg_price_m2_per_city,Stadt,price_m2,constructionyear_range,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r,courtyard_estate,garage,house_auction,house_layout,house_viewing,multi_dwelling,renovated
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
115756801,True,True,False,False,2,normal,90,3598,Ranstadt,3240,very new,Bungalow,518500,A+,Erstbezug,3.0,160,1200.0,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,NaN,NaN,0,260,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.00,0.00,0.00,0.00,10.38,0.00,1.15,0.00,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.00,1.15,0.00,0.38,0.00,0.00\r,0,0,0,0,0,0,0
116853306,True,False,False,False,3,expensive,122,1821,Nideggen,2234,very old,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4.0,135,860.0,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945.0,4902.0,5043.0,153.0,NaN,12,162,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.00,0.00,0.00,0.00,18.52,1.23,0.62,0.00,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00\r,0,0,0,0,0,0,0
116959865,True,False,False,False,3,expensive,167,1773,Pegau,2972,very new,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3.0,76,612.0,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,04523,12,48.8,6288.0,3068.0,3220.0,129.0,NaN,24,94,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.00,0.00,0.00,0.00,14.89,1.06,2.13,0.00,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.00,1.06,0.00,1.06,0.00,0.00\r,0,0,0,0,0,0,0
116953174,False,False,False,False,2,normal,100,1518,52428,1518,very new,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6.0,144,550.0,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,NaN,NaN,10,148,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.00,0.00,0.00,0.00,12.84,0.68,0.68,0.00,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.00,2.03,0.00,0.00,0.68,0.00\r,0,0,0,0,0,0,0
116953218,False,False,False,False,2,normal,90,2260,Storkow,2036,very old,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4.0,141,801.0,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,NaN,NaN,10,86,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.0

In [0]:
# DummyDF.to_excel('Immoscout_informal_features.xlsx')

# Checkpoint. Loading xlsx 

In [0]:
df = pd.read_excel("/content/drive/My Drive/Datasets/Immoscout_informal_features.xlsx", sep='\t', error_bad_lines=False, dtype='unicode', lineterminator='\n')
pd.options.display.max_columns = None

## Descriptives exploration

### preparing dataframe

In [0]:
df1 = df
df1[['light_flooded', 'garden', 'noisy_property', 'quiet_property']] = df1[['light_flooded', 'garden', 'noisy_property', 'quiet_property']].replace(to_replace=['True', 'False'], value=[1, 0])
df1 = df1.apply(pd.to_numeric, errors='coerce').fillna(df1)
df1

,ID,light_flooded,garden,noisy_property,quiet_property,price_class_num,price_class,perc_price_of_avg,avg_price_m2_per_city,Stadt,price_m2,constructionyear_range,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r,courtyard_estate,garage,house_auction,house_layout,house_viewing,multi_dwelling,renovated
0,115756801,1,1,0,0,2,normal,90,3598,Ranstadt,3240,very new,Bungalow,518500,A+,Erstbezug,3,160,1200,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,NaN,NaN,0,260,99,75.31,33.3,99,15.29,32.31,79.23,45,8.46,3.46,0,0,0.38,3.46,14.23,15,6.54,4.62,10.77,0,6.15,6.15,0,0,0,0,10.38,0,1.15,0,0,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0,1.92,8.08,1.15,2.69,2.31,2.31,0,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0,1.15,0,0.38,0,0,0,0,0,0,0,0,0
1,116853306,1,0,0,0,3,expensive,122,1821,Nideggen,2234,very old,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4,135,860,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945,4902,5043,153,NaN,12,162,46.46,80.34,24.22,89.37,13.5,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0,0,0,0,18.52,1.23,0.62,0,0,10.49,2.47,0.62,0,1.85,2.47,3.09,3.09,0.62,1.23,0,1.85,0,0.62,0,0.62,8.02,3.09,4.32,0,1.23,0,1.23,4.32,1.85,3.7,2.47,3.7,1.23,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,116959865,1,0,0,0,3,expensive,167,1773,Pegau,2972,very new,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3,76,612,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,4523,12,48.8,6288,3068,3220,129,NaN,24,94,95.21,77.64,17.77,99,11.75,28.72,77.66,43.62,15.96,4.26,0,0,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0,0,0,0,14.89,1.06,2.13,0,0,11.7,1.06,0,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0,1.06,0,1.06,0,0,5.32,1.06,2.13,0,2.13,1.06,0,3.19,1.06,3.19,1.06,6.38,0,0,0,1.06,0,1.06,0,0,0,0,0,0,0,0,0
3,116953174,0,0,0,0,2,normal,100,1518,52428,1518,very new,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6,144,550,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,NaN,NaN,10,148,93.6,95.48,2.04,99,13.45,37.16,76.35,45.95,13.51,9.46,0,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0,6.76,6.08,0,0,0,0,12.84,0.68,0.68,0,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0,0,0.68,4.05,0.68,0.68,1.35,0,13.51,5.41,6.76,0.68,0,0,0,5.41,2.03,4.73,2.03,3.38,0.68,0,0,2.03,0,0,0.68,0,0,0,0,0,0,0,0
4,116953218,0,0,0,0,2,normal,90,2260,Storkow,2036,very old,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4,141,801,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,NaN,NaN,10,86,99,51.14,1,17.39,4.53,43.02,51.16,18.6,2.33,0,0,0,0,2.33,2.33,8.14,2.33,0,9.3,0,0,0,0,0,0,0,1.16,0,0,0,0,23.26,0,0,0,1.16,1.16,3.49,0,0,0,0,0,0,0,0,0,19.77,18.6,1.16,0,0,0,0,0,0,5.81,1.16,2.33,1.16,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [0]:
dftest2 = df1
mean_col = dftest2.groupby(['Stadt'])['Kaufpreis'].count() # not resetting the index!
dftest2 = dftest2.set_index(['Stadt']) # make the same index here
dftest2['listings_per_city'] = mean_col
dftest2 = dftest2.reset_index() # to take the hierarchical index off again
# dftest2 = dftest2.rename(columns={'perc_price_of_avg': 'price_m2_ratio'})
dftest2.listings_per_city = pd.to_numeric(dftest2.listings_per_city, errors='coerce').fillna(0).astype(np.int64)
dftest2['Population_density'] = pd.to_numeric(dftest2['Population_density'], errors='coerce').fillna(0).astype(np.int64)
dftest2 = pd.concat([dftest2['listings_per_city'],dftest2.drop('listings_per_city',axis=1)], axis=1)
dftest2['marketheat_ratio'] = dftest2['listings_per_city'] / dftest2['Population_density'] *100
dftest2['marketheat_ratio'] = dftest2['marketheat_ratio'].replace([np.inf, -np.inf], np.nan)
dftest2['marketheat_ratio'] = pd.to_numeric(dftest2['marketheat_ratio'], errors='coerce').fillna(0).astype(np.int64)
dftest2 = pd.concat([dftest2['marketheat_ratio'],dftest2.drop('marketheat_ratio',axis=1)], axis=1)
dftest2

,marketheat_ratio,listings_per_city,Stadt,ID,light_flooded,garden,noisy_property,quiet_property,price_class_num,price_class,perc_price_of_avg,avg_price_m2_per_city,price_m2,constructionyear_range,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r,courtyard_estate,garage,house_auction,house_layout,house_viewing,multi_dwelling,renovated
0,0,8,Ranstadt,115756801,1,1,0,0,2,normal,90,3598,3240,very new,Bungalow,518500,A+,Erstbezug,3,160,1200,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,0,NaN,0,260,99,75.31,33.3,99,15.29,32.31,79.23,45,8.46,3.46,0,0,0.38,3.46,14.23,15,6.54,4.62,10.77,0,6.15,6.15,0,0,0,0,10.38,0,1.15,0,0,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0,1.92,8.08,1.15,2.69,2.31,2.31,0,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0,1.15,0,0.38,0,0,0,0,0,0,0,0,0
1,9,15,Nideggen,116853306,1,0,0,0,3,expensive,122,1821,2234,very old,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4,135,860,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945,4902,5043,153,NaN,12,162,46.46,80.34,24.22,89.37,13.5,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0,0,0,0,18.52,1.23,0.62,0,0,10.49,2.47,0.62,0,1.85,2.47,3.09,3.09,0.62,1.23,0,1.85,0,0.62,0,0.62,8.02,3.09,4.32,0,1.23,0,1.23,4.32,1.85,3.7,2.47,3.7,1.23,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13,18,Pegau,116959865,1,0,0,0,3,expensive,167,1773,2972,very new,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3,76,612,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,4523,12,48.8,6288,3068,3220,129,NaN,24,94,95.21,77.64,17.77,99,11.75,28.72,77.66,43.62,15.96,4.26,0,0,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0,0,0,0,14.89,1.06,2.13,0,0,11.7,1.06,0,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0,1.06,0,1.06,0,0,5.32,1.06,2.13,0,2.13,1.06,0,3.19,1.06,3.19,1.06,6.38,0,0,0,1.06,0,1.06,0,0,0,0,0,0,0,0,0
3,0,1,52428,116953174,0,0,0,0,2,normal,100,1518,1518,very new,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6,144,550,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,0,NaN,10,148,93.6,95.48,2.04,99,13.45,37.16,76.35,45.95,13.51,9.46,0,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0,6.76,6.08,0,0,0,0,12.84,0.68,0.68,0,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0,0,0.68,4.05,0.68,0.68,1.35,0,13.51,5.41,6.76,0.68,0,0,0,5.41,2.03,4.73,2.03,3.38,0.68,0,0,2.03,0,0,0.68,0,0,0,0,0,0,0,0
4,0,7,Storkow,116953218,0,0,0,0,2,normal,90,2260,2036,very old,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4,141,801,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,0,NaN,10,86,99,51.14,1,17.39,4.53,43.02,51.16,18.6,2.33,0,0,0,0,2.33,2.33,8.14,2.33,0,9.3,0,0,0,0,0,0,0,1.16,0,0,0,0,23.26,0,0,0,1.16,1.16,3.49,0,0,0,0,0,0,0,0,0,19.77,18.6,1.16,0,0,0,0,0,0,5.81,1.16,2.33,1.16,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [0]:
dftest2 = dftest2.set_index('ID')
dftest2

,marketheat_ratio,listings_per_city,Stadt,light_flooded,garden,noisy_property,quiet_property,price_class_num,price_class,perc_price_of_avg,avg_price_m2_per_city,price_m2,constructionyear_range,Typ,Kaufpreis,Energieeffizienzklasse,Objektzustand,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,Title,Beschreibung,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Average_income,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r,courtyard_estate,garage,house_auction,house_layout,house_viewing,multi_dwelling,renovated
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
115756801,0,8,Ranstadt,1,1,0,0,2,normal,90,3598,3240,very new,Bungalow,518500,A+,Erstbezug,3,160,1200,2020,Zeitlose Ästhetik im Bauhaus-Stil! Jetzt Onlin...,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,63691,4,NaN,NaN,NaN,NaN,0,NaN,0,260,99,75.31,33.3,99,15.29,32.31,79.23,45,8.46,3.46,0,0,0.38,3.46,14.23,15,6.54,4.62,10.77,0,6.15,6.15,0,0,0,0,10.38,0,1.15,0,0,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0,1.92,8.08,1.15,2.69,2.31,2.31,0,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0,1.15,0,0.38,0,0,0,0,0,0,0,0,0
116853306,9,15,Nideggen,1,0,0,0,3,expensive,122,1821,2234,very old,Einfamilienhaus (freistehend),301667,NaN,Erstbezug,4,135,860,0,Bauen statt Miete zahlen! Ihr neues Zuhause in...,Das LifeStyle 13.10 S bietet Ihnen und Ihrer F...,52385,4,65.04,9945,4902,5043,153,NaN,12,162,46.46,80.34,24.22,89.37,13.5,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0,0,0,0,18.52,1.23,0.62,0,0,10.49,2.47,0.62,0,1.85,2.47,3.09,3.09,0.62,1.23,0,1.85,0,0.62,0,0.62,8.02,3.09,4.32,0,1.23,0,1.23,4.32,1.85,3.7,2.47,3.7,1.23,0,0,0,0,0,0,0,0,0,0,0,0,0,0
116959865,13,18,Pegau,1,0,0,0,3,expensive,167,1773,2972,very new,Einfamilienhaus (freistehend),225900,A+,Erstbezug,3,76,612,2021,Wohnen und leben auf einer Ebene!,+++Gerne bietet Kern-Haus Ihnen eine umfangrei...,4523,12,48.8,6288,3068,3220,129,NaN,24,94,95.21,77.64,17.77,99,11.75,28.72,77.66,43.62,15.96,4.26,0,0,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0,0,0,0,14.89,1.06,2.13,0,0,11.7,1.06,0,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0,1.06,0,1.06,0,0,5.32,1.06,2.13,0,2.13,1.06,0,3.19,1.06,3.19,1.06,6.38,0,0,0,1.06,0,1.06,0,0,0,0,0,0,0,0,0
116953174,0,1,52428,0,0,0,0,2,normal,100,1518,1518,very new,Einfamilienhaus (freistehend),218659,NaN,Erstbezug,6,144,550,2020,Das neue LifeStyle 14.03S ! Auch als Stadtvill...,Jeder von uns hat Träume. Ein eigenes Haus zu ...,52428,12,NaN,NaN,NaN,NaN,0,NaN,10,148,93.6,95.48,2.04,99,13.45,37.16,76.35,45.95,13.51,9.46,0,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0,6.76,6.08,0,0,0,0,12.84,0.68,0.68,0,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0,0,0.68,4.05,0.68,0.68,1.35,0,13.51,5.41,6.76,0.68,0,0,0,5.41,2.03,4.73,2.03,3.38,0.68,0,0,2.03,0,0,0.68,0,0,0,0,0,0,0,0
116953218,0,7,Storkow,0,0,0,0,2,normal,90,2260,2036,very old,Einfamilienhaus (freistehend),287200,NaN,Erstbezug,4,141,801,0,"Barriere freier Bungalow in Kehrigk , jetzt ko...",Dieses Angebot beinhaltet:\n\n- Projektiertes ...,15859,12,NaN,NaN,NaN,NaN,0,NaN,10,86,99,51.14,1,17.39,4.53,43.02,51.16,18.6,2.33,0,0,0,0,2.33,2.33,8.14,2.33,0,9.3,0,0,0,0,0,0,0,1.16,0,0,0,0,23.26,0,0,0,1.16,1.16,3.49,0,0,0,0,0,0,0,0,0,19.77,18.6,1.16,0,0,0,0,0,0,5.81,1.16,2.33,1.16,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [0]:
df3 = dftest2
df3 = df3.apply(pd.to_numeric, errors='coerce')
df3 = df3.drop(['Stadt', 'Average_income'], 1)
df3 = df3.dropna(axis=1, how='all')
# df3 = df3.dropna()
df3

,marketheat_ratio,listings_per_city,light_flooded,garden,noisy_property,quiet_property,price_class_num,perc_price_of_avg,avg_price_m2_per_city,price_m2,Kaufpreis,Anzahl_zimmer,Wohnflaeche (in m²),Grundstueck (in m²),Baujahr,PLZ,timetosell,Flaeche_km2,Einwohner,Men,Women,Population_density,Topic,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you_total,ipron,article,prep,auxverb,adverb,conj,negate,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achiev,power,reward,risk,focuspast,focuspresent,focusfuture,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu\r,courtyard_estate,garage,house_auction,house_layout,house_viewing,multi_dwelling,renovated
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
115756801,0,8,1.0,1.0,0.0,0.0,2,90,3598,3240,518500,3.0,160,1200.0,2020,63691.0,4,NaN,NaN,NaN,NaN,0,0.0,260.0,99.00,75.31,33.30,99.00,15.29,32.31,79.23,45.00,8.46,3.46,0.00,0.00,0.38,3.46,14.23,15.00,6.54,4.62,10.77,0.00,6.15,6.15,0.0,0.0,0.0,0.0,10.38,0.00,1.15,0.0,0.00,13.08,2.31,1.15,1.15,2.69,1.54,3.46,2.31,1.15,0.77,0.38,4.23,0.38,0.77,0.00,1.92,8.08,1.15,2.69,2.31,2.31,0.00,0.38,3.46,1.92,4.23,5.38,6.54,0.38,0.38,0.0,1.15,0.0,0.38,0.00,0.0,0,0,0,0,0,0,0
116853306,9,15,1.0,0.0,0.0,0.0,3,122,1821,2234,301667,4.0,135,860.0,0,52385.0,4,65.04,9945.0,4902.0,5043.0,153,12.0,162.0,46.46,80.34,24.22,89.37,13.50,35.19,75.93,48.15,16.05,11.11,2.47,0.62,0.00,3.09,6.79,13.58,7.41,4.32,15.43,1.85,3.09,3.09,0.0,0.0,0.0,0.0,18.52,1.23,0.62,0.0,0.00,10.49,2.47,0.62,0.00,1.85,2.47,3.09,3.09,0.62,1.23,0.00,1.85,0.00,0.62,0.00,0.62,8.02,3.09,4.32,0.00,1.23,0.00,1.23,4.32,1.85,3.70,2.47,3.70,1.23,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0,0,0,0,0,0,0
116959865,13,18,1.0,0.0,0.0,0.0,3,167,1773,2972,225900,3.0,76,612.0,2021,4523.0,12,48.80,6288.0,3068.0,3220.0,129,24.0,94.0,95.21,77.64,17.77,99.00,11.75,28.72,77.66,43.62,15.96,4.26,0.00,0.00,1.06,7.45,10.64,14.89,2.13,1.06,13.83,2.13,7.45,7.45,0.0,0.0,0.0,0.0,14.89,1.06,2.13,0.0,0.00,11.70,1.06,0.00,1.06,4.26,1.06,4.26,3.19,2.13,1.06,0.00,1.06,0.00,1.06,0.00,0.00,5.32,1.06,2.13,0.00,2.13,1.06,0.00,3.19,1.06,3.19,1.06,6.38,0.00,0.00,0.0,1.06,0.0,1.06,0.00,0.0,0,0,0,0,0,0,0
116953174,0,1,0.0,0.0,0.0,0.0,2,100,1518,1518,218659,6.0,144,550.0,2020,52428.0,12,NaN,NaN,NaN,NaN,0,10.0,148.0,93.60,95.48,2.04,99.00,13.45,37.16,76.35,45.95,13.51,9.46,0.00,4.05,2.03,3.38,8.11,15.54,7.43,4.05,9.46,0.00,6.76,6.08,0.0,0.0,0.0,0.0,12.84,0.68,0.68,0.0,0.68,10.14,0.68,1.35,0.68,0.68,2.03,0.68,0.68,0.00,0.00,0.68,4.05,0.68,0.68,1.35,0.00,13.51,5.41,6.76,0.68,0.00,0.00,0.00,5.41,2.03,4.73,2.03,3.38,0.68,0.00,0.0,2.03,0.0,0.00,0.68,0.0,0,0,0,0,0,0,0
116953218,0,7,0.0,0.0,0.0,0.0,2,90,2260,2036,287200,4.0,141,801.0,0,15859.0,12,NaN,NaN,NaN,NaN,0,10.0,86.0,99.00,51.14,1.00,17.39,4.53,43.02,51.16,18.60,2.33,0.00,0.00,0.00,0.00,2.33,2.33,8.14,2.33,0.00,9.30,0.00,0.00,0.00,0.0,0.0,0.0,0.0,1.16,0.00,0.00,0.0,0.00,23.26,0.00,0.00,0.00,1.16,1.16,3.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19.77,18.60,1.16,0.00,0.00,0.00,0.00,0.00,0.00,5.81,1.16,2.33,1.16,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37000048,2,2,0.0,1.0,0.0,0.0,3,175,642,1127,239000,10.0,212,646.0,0,98559.0,12,23.40,1608.0,788.0,820.0,69,18.0,70.0,98.84,70.47,4.13,52.93,35.00,38.57,54.29,27.14,4.29,0.00,0.00,0.00,0.00,2.86,2.86,12.86,4.29,0.00,11.43,0.00,1.43,1.43,0.0,0.0,0.0,0.0,5.71,0.00,0.00,0.0,0.00,11.43,1.43,0.00,0.00,0.00,2.86,1.43,0.00

In [0]:
df3.to_csv('Immoscout_SPSS_dataset_integers.csv')